# Overview
- 実験をgoogle splead sheet で管理する方法についてまとめます。
- 参考URL
  > https://amalog.hateblo.jp/entry/kaggle-snippets
  
- 正則化変える

# Params

In [1]:
NB = '003'
DATA_NAME = 'cancer'
MODEL = 'logistic_regression'

# Import everything I need :)

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

# splead sheet
import json
import requests

# my_func

In [3]:
def write_spreadsheet(*args):
    endpoint = 'https://script.google.com/macros/s/AKfycbz_kjLNKyPv3-8VU6JzPrdHuWz4kP1xaCOhfSvx0oMR7QS3Ul4/exec'       # <-------- ここに取得したidを貼る
    requests.post(endpoint, json.dumps(args))

# 学習 ---> 評価

create datasets

In [4]:
cancer = load_breast_cancer()

X = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
y = cancer.target

<br>
std

In [5]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X = pd.DataFrame(X, columns=cancer.feature_names)

<br>
crate train, valid sets

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.5, random_state=42)

<br>
fit and pred

In [7]:
# model
model_params = {'penalty': 'l1', 'C': 1}
model = LogisticRegression(**model_params)

# fit
model.fit(X_train, y_train)

# pred
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<br>
eval

In [9]:
f1_train = f1_score(y_train, y_train_pred)
f1_valid = f1_score(y_valid, y_valid_pred)

print('f1 score')
print(f'train: {f1_train :.3f}')
print(f'valid: {f1_valid :.3f}')

f1 score
train: 0.985
valid: 0.981


# Google Splead Sheet

In [10]:
comment    = 'C を変更'

_f1_score  = f'{f1_valid:.4f}'
text       = [NB, DATA_NAME, model.__class__.__name__, model_params, _f1_score,str(X.columns.tolist()), comment]

write_spreadsheet(*text)